#### 데이터 분석

- 문제 정의 = 가장 적절한 고객 관리 타이밍은?
- 가설 설정 = 수강완료 수강생이 가장 많은날과 해당 날짜의 시간대별 수강시간을 알아보자
- 데이터 분석 기본 세팅
- 데이터 분석
- 분석 결과 시각화
- 최종 결론
  - 독려 메세지의 가장 적절한 타이밍을 찾기 위해 수강생들의 접속 일자 및 시간을 살펴보았는데요,
  - 먼저 요일을 살펴보자면 아래 보시는 것과 같이, 일주일 중 화요일 그리고 일요일에 수강이 많이 이뤄졌고,
  - 금요일 과 수요일에 수강 하는 인원이 가장 적었습니다.
  - 이어서, 일별 접속 시간의 추이를 살펴보았는데요
  - 저녁시간 쯤인 18시대에 접속이 폭발적으로 많았고, 밤 21시쯤에 감소하는 추세를 보였습니다.
  - 요일 별, 시간대별로도 살펴보았더니 역시 화요일 18시에 가장 많은 접속자가 있다는 것을 확인할 수 있었습니다.
  - 고객 관리 문자는 가장 수강을 많이 한 `화요일, 일요일 저녁시간`쯤에 독려 문자를
  - 가장 수강을 적게 한 `금요일, 수요일 오전`시간에 동기부여 문자를 보내보는 것은 어떨까요?

#### 좋은 가설이란?

- 풀고자 하는 문제의 방향성과 일치하는 가설 → 무슨 문제를 풀고자 하는지 명확히 알 것
- 테스트 가능한 가설 → 데이터로 확인이 가능한 가설을 세울 것
- 액션으로 이어질 수 있는 가설 → 가설 검증 후 결과에 따라 특정 액션으로 유도될 수 있을 것

In [ ]:
# Pandas 라이브러리 불러오기

import pandas as pd

# 수강생 접속 파일 불러오기

sparta_data = pd.read_table('../Data/access_detail.csv',sep=',')
sparta_data.head()

In [ ]:
# type함수를 사용해 access_date는 문자열로 되어있음을 확인할 수 있다

print(type(sparta_data['access_date'][0]))

In [ ]:
# 문자열 데이터를 시계열 데이터로 변경 후 파생변수 생성

sparta_data['access_date_time'] = pd.to_datetime(sparta_data['access_date'], format='ISO8601')
sparta_data.tail(5)

In [ ]:
# 해당 날짜의 요일 추출 후 파생변수 생성

sparta_data['access_date_time_weekday'] = sparta_data['access_date_time'].dt.day_name()
sparta_data['access_date_time_hour'] = sparta_data['access_date_time'].dt.hour
sparta_data.tail(5)

In [ ]:
# 요일별 수강생 합계

weeks = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekdata = sparta_data.groupby('access_date_time_weekday')['user_id'].count()
weekdata

In [ ]:
# week의 리스트에 따라 데이터들을 다시한번 재배열 할수 있어요!

weekdata = weekdata.agg(weeks)
weekdata

In [ ]:
# 해당 날짜의 시간대 별 수강생 합계 

hourdata = sparta_data.groupby('access_date_time_hour')['user_id'].count()
hourdata = hourdata.sort_index() # 인덱스를 기준으로 오름차순 정렬
hourdata

In [ ]:
# 시각화 라이브러리 불러오기

import matplotlib.pyplot as plt
import numpy as np

# 글씨 깨짐방지

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 폰트 설정

plt.rc('font', family='Malgun Gothic')

In [ ]:
# 요일별 수강생 수 바 그래프 그리기
# 그래프 사이즈

plt.figure(figsize=(10,5))

# 그래프 x축 y축

plt.bar(weekdata.index, weekdata)

# 그래프 명

plt.title('요일별 수강 완료 수강생 수')

# 그래프 x축 레이블

plt.xlabel('요일')

# 그래프 y축 레이블

plt.ylabel('수강생(명)')

# x축 레이블을 90도로 변환 

plt.xticks(rotation=90)

# 그래프 출력

plt.show()

In [ ]:
# 시간 별 접속 하는 수강생 수의 라인 그래프 그리기
# 그래프 사이즈 변경

plt.figure(figsize=(10,5))

# 그래프 x축 y축

plt.plot(hourdata.index, hourdata)

# 그래프 명

plt.title('시간별 수강 완료 사용자 수')

# 그래프 x축 레이블

plt.xlabel('시간')

# 그래프 y축 레이블

plt.ylabel('사용자(명)')

# x축 눈금 표시 하기

plt.xticks(np.arange(24))

# 그래프 출력

plt.show()

In [ ]:
# 히트맵을 만들기 위한 피벗테이블 만들기

# values : 열에 들어 가는 부분
# index : 행에 들어가는 부분
# aggfunc : 데이터 축약시 사용할 함수

sparta_data_pivot_table = pd.pivot_table(sparta_data, values='user_id', 
                       index=['access_date_time_weekday'],
                       columns=['access_date_time_hour'], 
                       aggfunc="count").agg(weeks)
sparta_data_pivot_table

In [ ]:
# 히트맵 만들기

# 그래프 사이즈 변경

plt.figure(figsize=(14,5))

# pcolor를 이용하여 heatmap 그리기

plt.pcolor(sparta_data_pivot_table)

# 히트맵에서의 x축

plt.xticks(np.arange(0.5, len(sparta_data_pivot_table.columns), 1), sparta_data_pivot_table.columns)

# 히트맵에서의 y축

plt.yticks(np.arange(0.5, len(sparta_data_pivot_table.index), 1), sparta_data_pivot_table.index)

# 그래프 명

plt.title('요일별 종료 시간 히트맵')

# 그래프 x축 레이블

plt.xlabel('시간')

# 그래프 y축 레이블

plt.ylabel('요일')

# plt.colorbar() 명령어를 추가하면 그래프 옆에 숫자별 색상값을 나타내는 컬러바를 보여 줍니다

plt.colorbar()
plt.show()